Fotmob selenium webscraper:

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

In [40]:
# Setting up driver and URL
DRIVER_PATH = 'C:/Users/micha/OneDrive/Pulpit/chromedriver/chromedriver.exe' #Your chromedriver path
options = Options()
options.headless = True
service = Service(executable_path=DRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.fotmob.com/leagues/47') #Replace the number after /leagues/ with fotmobs league id that you want to scrape - in this instance, id=47 corresponds to Premier League

#Note: the scraper can be expanded to scrape different seasons as well - if you want to do so, include selecting season from a dropdown in top right corner in the code below or add for example ?season=2021-2022 at the end of 
#fotmob link in driver.get() command above

#Process of obtaining stat specific links 
consent = driver.find_element(By.XPATH, "//button[contains(@class, 'fc-button') and contains(@class, 'fc-cta-consent') and contains(@class, 'fc-primary-button')]")
consent.click() #Getting cookies consent out of the way

screen_height = driver.execute_script("return window.screen.height;")
driver.execute_script("window.scrollTo(0, {screen_height}*0.1);".format(screen_height=screen_height))
time.sleep(5) #Scrolling to the top of the page

stats = driver.find_elements(By.XPATH, "//a[contains(@class, 'css-1wpn8in-NavLinkCSS-applyMediumHover-TabIndicator') and contains(@class, 'eyuy2lc0')]")
stats[1].click()
time.sleep(5) #Clicking on the stats button, if you want to scrape other sections, feel free to change the xpath

stats_teams = driver.find_element(By.XPATH, "//a[contains(@class, 'css-dqp8ld-StyledTabLink-applyMediumHover-TabIndicator') and contains(@class, 'e1az4uno0')]")
stats_teams.click() #Choosing team specific stats
time.sleep(5)

# Getting the categories container
categories = driver.find_elements(By.XPATH, '//*[contains(@class, "css-11hkooy-TLStatsTopThreeCSS") and contains(@class, "ecfii2615")]')

# Extracting and storing the stat specific links
links = []
for category in categories:
    link = category.find_element(By.TAG_NAME, 'a').get_attribute('href')
    links.append(link)

df=pd.DataFrame() #Declaring dataframe that will contain our final results

#This loop is going to iterate through obtained links and scrape respective data
for link in links:
    
    driver.get(link)

    #Lists that will store club and stats data
    values = []
    clubs = []

    #Extracting club and stat names for the first page (10 rows)
    values_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'css-wo228r-StatValue') and contains(@class, 'eybz1nb7')]")
    clubs_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'css-cfc8cy-TeamOrPlayerName') and contains(@class, 'eybz1nb9')]")

    #Convering obtained values to text
    values.extend([element.text for element in values_elements])
    clubs.extend([element.text for element in clubs_elements])
    time.sleep(5)

    #Clicking on the dynamic next button - please note that it's not reffering to button class, with this approach the button wasn't working 
    next_button = driver.find_element(By.XPATH, "//button[./span/text()='Next']")
    next_button.click()
    time.sleep(5)

    #Same procedure of obtaining data, but now for the second page (Usually 8 or 10 rows depending on the league - if it has more than 20 teams just copy next button click and the extracting procedure below)
    values_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'css-wo228r-StatValue') and contains(@class, 'eybz1nb7')]")
    clubs_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'css-cfc8cy-TeamOrPlayerName') and contains(@class, 'eybz1nb9')]")

    #Convering obtained values to text
    values.extend([element.text for element in values_elements])
    clubs.extend([element.text for element in clubs_elements])

    #Creating stat specific column names based on links
    nazwa= link.split('/')
    nazwa = nazwa[-2]
    nazwa= nazwa.replace('_',' ') 
    time.sleep(1) 

    #Creating temporary dataframe that will either be a foundation of final dataframe or will be left joined to it
    temp_df=pd.DataFrame({'club': clubs, nazwa: values})

    if link==links[0]: #Only the first stat will be used as a df foundation, the rest will be left joined
        df=temp_df
    else:
        df=pd.merge(df, temp_df, how='left', on='club')
    df.head()
    time.sleep(5)

#Close the driver after the scraping
driver.quit()
    

In this instance, I've scraped Top5 leagues stats for the 2023/2024 season and saved it to the respective csv files - the process below is just merging them together and making column titles more readable

In [42]:
ENG = pd.read_csv("C:/Users/micha/Downloads/ENGscrap.csv")
ESP = pd.read_csv("C:/Users/micha/Downloads/ESPscrap.csv")
ITA = pd.read_csv("C:/Users/micha/Downloads/ITAscrap.csv") 
GER = pd.read_csv("C:/Users/micha/Downloads/GERscrap.csv") 
FRA = pd.read_csv("C:/Users/micha/Downloads/FRAscrap.csv")
ENG.insert(0, 'League', 'ENG')
ESP.insert(0, 'League', 'ESP')
ITA.insert(0, 'League', 'ITA')
GER.insert(0, 'League', 'GER')
FRA.insert(0, 'League', 'FRA')
df = pd.concat([ENG, ESP, ITA, GER, FRA])
df.drop(columns=['Unnamed: 0'], inplace=True)
df.rename(columns={'League':'league', 'klub':'club', 'rating team':'team rating', 'goals team match':'goals per match', 'goals conceded team match':'conceded goal per match', 
                   'possession percentage team':'average possesion', 'clean sheet team':'clean sheets', ' expected goals team':'expected goals', 'ontarget scoring att team':'shots on target per match', 
                   ' big chance team':'big chances', 'big chance missed team':'big chances missed', 'accurate pass team':'accurate passes per match', 'accurate long balls team':'accurate long balls per match', 
                   'accurate cross team':'accurate crosses per match', 'penalty won team':'penalities awarded', 'touches in opp box team':'touches in opposition box', 'corner taken team':'corners', 
                   'expected goals conceded team':'expected goals conceded', 'interception team':'interceptions per match', 'won tackle team':'succesful tackles per match', 'effective clearance team':'clearances per match', 
                   'poss won att 3rd team':'possesion won final 3rd per match', 'penalty conceded team':'penalties conceded', 'saves team':'saves per match', 'fk foul lost team':'fouls per match',
                   'total yel card team':'total yellow cards', 'total red card team':'red cards'}, inplace=True)
df.to_csv("DaneSelenium.csv")

In [2]:
df = pd.read_csv("C:/Users/micha/Downloads/DaneSelenium.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)

Creating an interactive dashboard that will display scraped data:

In [11]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
from dash_bootstrap_templates import load_figure_template
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np

load_figure_template("darkly") #dashboards theme

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Checklist(
                options=[{'label': i, 'value': i} for i in df['league'].unique()], #leagues checklist
                value=['ENG', 'ESP', 'ITA', 'GER', 'FRA'],
                id='league-checklist',
                style={'color': 'white'}
            )
        ], style={'width': '65%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                options=[{'label': col, 'value': col} for col in df.columns[2:]], #dropdown to select analysed stats, first two columns aren't included as they're only text
                value=df.columns[2],
                id='xaxis-dropdown'
            )
        ], style={'width': '35%', 'float': 'right', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                options=[{'label': col, 'value': col} for col in df.columns[2:]],
                value=df.columns[3],
                id='yaxis-dropdown'
            )
        ], style={'width': '35%', 'float': 'right', 'display': 'inline-block', 'margin-top': '-55px'})
    ], style={'padding': '10px 5px'}),

    #Graphs positioning

    dcc.Graph(id='scatter-plot'),

    html.Div([
        html.Div([
            dcc.Graph(id='histogram-1')
        ], style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Graph(id='histogram-2')
        ], style={'width': '49%', 'display': 'inline-block'})
    ], style={'padding': '10px 5px'}),

    html.Div([
        html.Div([
            dcc.Graph(id='distplot-1')
        ], style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Graph(id='distplot-2')
        ], style={'width': '49%', 'display': 'inline-block'})
    ], style={'padding': '10px 5px'}),

    html.Div([
        html.Div([
            dcc.Graph(id='boxplot-1')
        ], style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Graph(id='boxplot-2')
        ], style={'width': '49%', 'display': 'inline-block'})
    ], style={'padding': '10px 5px'})
])

@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('histogram-1', 'figure'),
     Output('histogram-2', 'figure'),
     Output('distplot-1', 'figure'),
     Output('distplot-2', 'figure'),
     Output('boxplot-1', 'figure'),
     Output('boxplot-2', 'figure')],
    [Input('league-checklist', 'value'),
     Input('xaxis-dropdown', 'value'),
     Input('yaxis-dropdown', 'value')]
)
def update_plots(selected_leagues, xaxis_col, yaxis_col):
    filtered_df = df[df['league'].isin(selected_leagues)]
    filtered_df.dropna(subset=[xaxis_col, yaxis_col], inplace=True) #Select only data from chosen leagues and remove any possible NAs to avoid errors

    #Displayed plots with provided data as arguments 

    scatter_fig = px.scatter(filtered_df, x=xaxis_col, y=yaxis_col, color='league', hover_data=['club'])
    scatter_fig.update_layout(title=f'{xaxis_col} and {yaxis_col} comparison')

    histogram_1_fig = px.histogram(filtered_df, x=xaxis_col, color='league', opacity=0.7, barmode='group')
    histogram_1_fig.update_layout(title=f'Histogram of {xaxis_col}')

    histogram_2_fig = px.histogram(filtered_df, x=yaxis_col, color='league', opacity=0.7, barmode='group')
    histogram_2_fig.update_layout(title=f'Histogram of {yaxis_col}')

    distplot_1_fig = ff.create_distplot([filtered_df[filtered_df['league'] == league][xaxis_col] for league in selected_leagues], selected_leagues, show_hist=False, show_rug=False)
    distplot_1_fig.update_layout(title_text=f'Distplot of {xaxis_col}')

    distplot_2_fig = ff.create_distplot([filtered_df[filtered_df['league'] == league][yaxis_col] for league in selected_leagues], selected_leagues, show_hist=False, show_rug=False)
    distplot_2_fig.update_layout(title_text=f'Distplot of {yaxis_col}')

    boxplot_1_fig = px.box(filtered_df, y=xaxis_col, color='league')
    boxplot_1_fig.update_layout(title=f'Boxplot of {xaxis_col}')

    boxplot_2_fig = px.box(filtered_df, y=yaxis_col, color='league')
    boxplot_2_fig.update_layout(title=f'Boxplot of {yaxis_col}')

    return scatter_fig, histogram_1_fig, histogram_2_fig, distplot_1_fig, distplot_2_fig, boxplot_1_fig, boxplot_2_fig

if __name__ == '__main__':
    app.run_server(debug=True)
